## Hotel Demand Forecasting - preprocessing

Given two hotels (H1) - a resort hotel and (H2) - a city hotel, pertaining to actual hotel bookings, this project tr

[Article for context on hotel booking data](https://www.sciencedirect.com/science/article/pii/S2352340918315191)

### 1. Importing Necessary Libraries

In [91]:
import pandas as pd
import numpy as np

In [109]:
resort_h1 = pd.read_csv('H1.csv')
hotel_h2 = pd.read_csv('H2.csv')

### 2. Data Exploration

In [111]:
resort_h1.sample(8)

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
38599,0,207,2017,July,28,15,2,6,2,1,...,No Deposit,250,NULL,0,Transient,209.85,0,3,Check-Out,2017-07-23
27105,0,24,2016,August,34,17,0,3,2,0,...,No Deposit,250,NULL,0,Transient,200.10,0,2,Check-Out,2016-08-20
4171,0,117,2016,February,9,22,3,7,1,0,...,No Deposit,2,NULL,0,Contract,24.50,0,1,Check-Out,2016-03-03
29387,0,34,2016,October,44,27,0,0,2,0,...,No Deposit,6,NULL,0,Transient,0.00,0,0,Check-Out,2016-10-27
8596,1,383,2016,October,41,6,1,3,2,0,...,No Deposit,315,NULL,0,Transient-Party,48.00,0,0,Canceled,2016-03-04
37923,0,8,2017,June,26,27,1,5,2,0,...,No Deposit,240,NULL,0,Transient,146.67,0,1,Check-Out,2017-07-03
27559,0,219,2016,August,36,29,1,3,2,0,...,No Deposit,177,NULL,0,Transient,64.80,0,1,Check-Out,2016-09-02
7525,1,360,2016,August,33,13,2,2,2,0,...,No Deposit,240,NULL,0,Transient,113.40,0,2,Canceled,2015-08-31


### 2.1 Size and shape of the datasets

In [93]:
data = [resort_h1, hotel_h2]

[d.shape for d in data]

[(40060, 31), (79330, 31)]

**Resort (H1)**
- 40,060 rows of booking data
- 31 attributes/columns

**Hotel (H2)**
- 79,330 rows of booking data
- 31 attributes/columns

### 2.2 Information and datatypes of each attribute

In [95]:
[d.info() for d in data]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40060 entries, 0 to 40059
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IsCanceled                   40060 non-null  int64  
 1   LeadTime                     40060 non-null  int64  
 2   ArrivalDateYear              40060 non-null  int64  
 3   ArrivalDateMonth             40060 non-null  object 
 4   ArrivalDateWeekNumber        40060 non-null  int64  
 5   ArrivalDateDayOfMonth        40060 non-null  int64  
 6   StaysInWeekendNights         40060 non-null  int64  
 7   StaysInWeekNights            40060 non-null  int64  
 8   Adults                       40060 non-null  int64  
 9   Children                     40060 non-null  int64  
 10  Babies                       40060 non-null  int64  
 11  Meal                         40060 non-null  object 
 12  Country                      39596 non-null  object 
 13  MarketSegment   

[None, None]

### 2.3 Checking for missing data

In [94]:
def missing_func(df):
    return (df
            .isnull().sum().to_frame()
            .rename(columns={0: 'Missing_NACount'})
            .query('Missing_NACount > 0'))

[missing_func(d) for d in data]

[         Missing_NACount
 Country              464,
           Missing_NACount
 Children                4
 Country                24]

**Resort (H1)**
- 464 rows of data without a 'Country' of origin declared in the booking

**Hotel (H2)**
- 24 rows of data without a 'Country' of origin declared in the booking
- 4 rows of data without number of 'Children' declared in the booking

### 2.3.1 Handling missing data

We will handle missing country rows by imputing the NA values with 'Unknown'. Assuming that customers who did not provide a value for No. of Children meant that they are not bringing along any children during the booking, we will replace NA values in the Children column with the value 0.

In [77]:
# Handling missing country attribute
for d in data:
    d['Country'] = d['Country'].replace(np.nan, 'Unknown', regex=True)

print('NA/Missing Countries (Resort):', resort_h1.Country.isna().sum())
print('NA/Missing Countries (Hotel):', hotel_h2.Country.isna().sum())

# Handling missing children attribute
hotel_h2['Children'] = hotel_h2.Children.fillna(0)

print('\nNA/Missing Children Count (Hotel):', hotel_h2.Children.isna().sum())

NA/Missing Countries (Resort): 0
NA/Missing Countries (Hotel): 0

NA/Missing Children Count (Hotel): 0


In [85]:
resort_set = set(resort_h1.columns)
hotel_set = set(hotel_h2.columns)

list(resort_set.difference(hotel_set))

[]

### 2.3.2 Handling whitespaces in the data

Given that there are trailing whitespaces in some of the columns within the dataframe (observed by viewing inside the csvs), namely **Meal, ReservedRoomType, AssignedRoomType and DespositType**. We will further clean up the data by striping any whitespaces

In [104]:
# resort dataframe cleanup
resort_h1.Meal = resort_h1.Meal.str.strip()
resort_h1.ReservedRoomType = resort_h1.ReservedRoomType.str.strip()
resort_h1.AssignedRoomType = resort_h1.AssignedRoomType.str.strip()
resort_h1.DepositType = resort_h1.DepositType.str.strip()

# city hotel dataframe cleanup
hotel_h2['Meal'] = hotel_h2['Meal'].str.strip()
hotel_h2['ReservedRoomType'] = hotel_h2['ReservedRoomType'].str.strip()
hotel_h2['AssignedRoomType'] = hotel_h2['AssignedRoomType'].str.strip()
hotel_h2['DepositType'] = hotel_h2['DepositType'].str.strip()

In [108]:
hotel_h2[['Meal', 'ReservedRoomType', 'AssignedRoomType', 'DepositType']].iloc[0:1, :]

,Meal,ReservedRoomType,AssignedRoomType,DepositType
0,HB,A,A,No Deposit


### 3. Exporting cleaned datasets

In [112]:
resort_h1.to_csv('H1_cleaned.csv', index=False)
hotel_h2.to_csv('H2_cleaned.csv', index=False)